# Getting Courses
---

Set username and password if you want to login automatically. Not required.

In [1]:
USERNAME = ""
PASSWORD = ""

Log into temple and wait for the request session to be created.

In [2]:
from temple.planner import TUPlanner

planner = TUPlanner(username=USERNAME, password=PASSWORD)

Once the session has been created, execute the cell below to get the code for the term you are planning course for.

In [ ]:
planner.print_terms(3)

Once you have retrieved the term code, update the `TERM_CODE` variable with the code and specify which courses you are planning to take.

In [4]:
TERM_CODE = 202503

# Plotting Courses
---

In [5]:
SELECTED_COURSES = ["MATH2043", "IH0851", "MATH2101", "CIS1068"] # "CIS2033"
# SELECTED_COURSES = ["CIS1057", "CIS1166", "ENG0802", "CIS1001", "SCTC1001"]

In [6]:

import json


planner.select_term(TERM_CODE)
planner.select_courses(SELECTED_COURSES)
# planner.filter_sections({
#     "MATH2101": ["001"]
# })
# planner.filter_teachers({
#     "MATH2101": ["Erik Wallace"]
# })

# planner.set_filter(lambda s: s["sequenceNumber"] == "001")

schedule_combos = planner.get_combinations()

# print(json.dumps(schedule_combos, indent=4))

# print(json.dumps(a.get_schema(), indent=4))

# print(a.get_teacher("VGVhY2hlci0xMDA4OTk="))

# print(json.dumps(a.get_schools("Temple University"), indent=4))
# print(json.dumps(a.get_teachers("John Davies", "U2Nob29sLTk5OQ=="), indent=4))
# print(json.dumps(a.get_teacher_rating("VGVhY2hlci0xMDA4OTk="), indent=4))


# for teacher in a.get_teachers("John Davies", "U2Nob29sLTk5OQ==", offset=-1):
#     print(teacher)

In [ ]:
from temple.schedule import Schedule, ScheduleCompare

schedules = set(Schedule(schedule) for schedule in schedule_combos)
schedules = sorted(filter(ScheduleCompare.not_overlap, schedules), key=lambda x: (100000 - ScheduleCompare.teacher_rating(x, penalty_rating=0.0, penalty_num_ratings=100.0), ScheduleCompare.between_total(x)))
# schedules = list(schedules)[:5]
print(len(schedules))

In [ ]:
for i in range(len(schedules)):
    print(i)
    found = set()
    for class_type in dict(sorted(schedules[i].classes.items(), key=lambda t: t[0])).values():
        for _class in sorted(class_type, key=lambda c: c.subject_course):
            if _class.subject_course not in found:
                found.add(_class.subject_course)
                teachers = _class.get_teachers()
                if len(teachers) == 0:
                    print("  ", _class.subject_course, _class.section, "Teacher(Unknown)")
                else:
                    for _teacher in teachers:
                            print("  ", _class.subject_course, _class.section, _teacher)
    print("  ", "RATING:", ScheduleCompare.teacher_rating(schedules[i], penalty_rating=0.0, penalty_num_ratings=100.0))

In [ ]:
from IPython.display import Markdown, display
import json
from types import SimpleNamespace

def render_table(headers: list[str], rows: list[list[str]]):
    # Create the header row
    md = "| " + " | ".join(headers) + " |\n"
    md += "| " + " | ".join(["---"] * len(headers)) + " |\n"  # Separator row

    # Add each row of data
    for row in rows:
        # Replace newline characters with <br> for Markdown rendering
        sanitized_row = [cell.replace("\n", "<br>") for cell in row]
        md += "| " + " | ".join(sanitized_row) + " |\n"

    display(Markdown(md))

test = None

for schedule in schedules:
    print(
        f"WEEK_RANGE: {ScheduleCompare.week_range(schedule):.2f} hrs, "
        f"WEEK_TOTAL: {ScheduleCompare.week_total(schedule):.2f} hrs, "
        f"BREAK_TOTAL: {ScheduleCompare.between_total(schedule):.2f} hrs"
    )

    rows = []

    def a(b):
        print(SimpleNamespace(**b))
        return SimpleNamespace(**b)

    for cls in {_class.subject_course: _class for _class in [*schedule.classes["normal"], *schedule.classes["online"]]}.values():
        # print(json.loads(json.dumps(cls.info), object_hook=a))
        test = json.dumps(cls.info, indent=4)
        print(json.dumps(cls.info, indent=4))
        rows.append(
            (
                cls.subject_course,
                cls.section,
                ("\n").join([f"{teacher.first_name} {teacher.last_name}" for teacher in cls.get_teachers()]),
                ("\n").join([f"{teacher.avg_rating_rounded:.2f}" for teacher in cls.get_teachers()]),
                "In-person" if cls.info["instructionalMethod"] == "CLAS" else "Online",
                str(cls.info["creditHourLow"]),
            )
        )

    render_table(["Class", "Section", "Teachers", "Ratings", "Type", "Credits"], rows)

    schedule.show()
    break

In [38]:
import json
from types import SimpleNamespace
import re

def to_snake_case(key):
    # Convert CamelCase or mixedCase to snake_case
    return re.sub(r'(?<!^)(?=[A-Z])', '_', key).lower()

def custom_object_hook(d):
    # Transform keys to snake_case and create a SimpleNamespace
    return SimpleNamespace(**d)

# Example JSON string
# data = '{"name": "Alice", "age": 30, "cityName": "Wonderland"}'

print(test)

# Parse JSON with custom object_hook
namespace_obj = json.loads(test, object_hook=custom_object_hook)

print(namespace_obj.subjectCourse)      # Output: Alice
print(namespace_obj.sectionAttributes[0].isZTCAttribute)       # Output: 30


{
    "id": 855632,
    "term": "202503",
    "termDesc": "2025 Spring",
    "courseReferenceNumber": "46043",
    "partOfTerm": "1",
    "courseNumber": "0851",
    "subject": "IH",
    "subjectDescription": "Intellectual Heritage",
    "sequenceNumber": "710",
    "campusDescription": "Main",
    "scheduleTypeDescription": "Base Lecture",
    "courseTitle": "Intellectual Heritage I: The Good Life",
    "creditHours": null,
    "maximumEnrollment": 28,
    "enrollment": 11,
    "seatsAvailable": 17,
    "waitCapacity": 5,
    "waitCount": 0,
    "waitAvailable": 5,
    "crossList": null,
    "crossListCapacity": null,
    "crossListCount": null,
    "crossListAvailable": null,
    "creditHourHigh": null,
    "creditHourLow": 3,
    "creditHourIndicator": null,
    "openSection": true,
    "linkIdentifier": null,
    "isSectionLinked": false,
    "subjectCourse": "IH0851",
    "faculty": [
        {
            "bannerId": "131742",
            "category": null,
            "class": "n